In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf


In [2]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
sess = tf.InteractiveSession()

In [4]:
def Weight_variables(shape):
    initial = tf.truncated_normal(shape, stddev=0.1) #stddev是标准差, 返回充斥截断正态分布值的tensor,shape是输入
    return tf.Variable(initial) #定义这个tensor为variable

def bias_variables(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


In [5]:
def conv2d(x,W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME') #W其实是kernel.也其实是权重,名字没那么重要了.

def max_pool_2x2(x):
    #返回一个Tensor，类型不变，shape仍然是[batch, height, width, channels]这种形式
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1, 2, 2, 1], padding='SAME')

In [6]:
# input x , y_ for true labels
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None,10])

x_image = tf.reshape(x, [-1, 28, 28, 1]) # -1是代表样本数量不变, 1代表1个颜色通道.


In [7]:

#定义卷积
# 第一层

W_conv1 = Weight_variables([5,5,1,32]) # 5x5 的kernel, 1个颜色通道(gray),其实输入也就是1层, 32个卷积核,对应的会提取32个特征
b_conv1 = bias_variables([32]) # 对应32个特征
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) # wx+b ,  relu ,  
h_pool1 = max_pool_2x2(h_conv1)


#### 第二层的输入
[5,5,32,64] 的 32 是第一层的输出的特征点

In [8]:
# 定义第二层
W_conv2 = Weight_variables([5,5,32,64]) # 32个输入特征,64个输出特征
b_conv2 = bias_variables([64])
h_conv2 = tf.nn.relu((conv2d(h_pool1,W_conv2) + b_conv2))
h_pool2 = max_pool_2x2(h_conv2)

#### 全连接层
两次max pooling之后,deci了两次, 28x28 --> 7x7了.
第二层输出64个单位.

In [9]:
# 定义全连接层
W_fc1 = Weight_variables([7*7*64, 1024]) #隐含节点是1024个
b_fc1 = bias_variables([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64]) #把第二层给平坦化
h_fc1 = tf.nn.relu( tf.matmul(h_pool2_flat, W_fc1) + b_fc1)



In [10]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

In [11]:
#经sigmoid得到概率表达
W_fc2 = Weight_variables([1024, 10])
b_fc2 = bias_variables([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [12]:
#损失函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv),reduction_indices=[1]))
train_step = tf.train.AdadeltaOptimizer(1e-4).minimize(cross_entropy)


In [13]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [14]:
tf.global_variables_initializer().run()
for i in range(20000):
    batch =  mnist.train.next_batch(50)
    if i%100 == 0:
        train_acc = acc.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})
        print("step, train acc", i, train_acc)
    
    train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})

step, train acc 0 0.1
step, train acc 100 0.08
step, train acc 200 0.04
step, train acc 300 0.1
step, train acc 400 0.04
step, train acc 500 0.1
step, train acc 600 0.02
step, train acc 700 0.08
step, train acc 800 0.04
step, train acc 900 0.06
step, train acc 1000 0.08
step, train acc 1100 0.06
step, train acc 1200 0.0
step, train acc 1300 0.12
step, train acc 1400 0.04
step, train acc 1500 0.06
step, train acc 1600 0.12
step, train acc 1700 0.08
step, train acc 1800 0.04
step, train acc 1900 0.04
step, train acc 2000 0.06
step, train acc 2100 0.04
step, train acc 2200 0.02
step, train acc 2300 0.06
step, train acc 2400 0.08
step, train acc 2500 0.06
step, train acc 2600 0.1
step, train acc 2700 0.06
step, train acc 2800 0.1
step, train acc 2900 0.02
step, train acc 3000 0.02
step, train acc 3100 0.06
step, train acc 3200 0.08
step, train acc 3300 0.06
step, train acc 3400 0.0
step, train acc 3500 0.06
step, train acc 3600 0.1
step, train acc 3700 0.06
step, train acc 3800 0.04
step, 

In [16]:
acc.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0})

0.2073